In [1]:
!pip install gradio
!pip install joblib
import gradio as gr
import numpy as np
import pandas as pd
import joblib
from google.colab import drive
drive.mount('/content/drive')

stacking_model = joblib.load('/content/drive/My Drive/Rtfp/models/stacked_model.pkl')
scaler = joblib.load('/content/drive/My Drive/Rtfp/models/scaler.pkl')

def predict_heart_disease(age, sex, cp, trestbps, chol, fbs, restecg, thalach,
                          exang, oldpeak, slope, ca, thal_3, thal_6, thal_7):
  try:
    if sum([thal_3, thal_6, thal_7]) != 1:
        return "Error: Select exactly one thal type."
    cp_1 = cp_2 = cp_3 = cp_4 = 0
    if cp == 1:
        cp_1 = 1
    elif cp == 2:
        cp_2 = 1
    elif cp == 3:
        cp_3 = 1
    elif cp == 4:
        cp_4 = 1

    input_data = np.array([[age, sex, thalach, exang, oldpeak, slope, ca, chol, fbs,
                            cp_1, cp_2, cp_3, cp_4, thal_3, thal_6, thal_7]])
    columns = ["age", "sex", "thalach", "exang", "oldpeak", "slope", "ca", "chol", "fbs",
               "cp_1.0", "cp_2.0", "cp_3.0", "cp_4.0", "thal_3.0", "thal_6.0", "thal_7.0"]

    input_df = pd.DataFrame(input_data, columns=columns)
    to_scale = ["age", "thalach", "oldpeak", "chol"]
    input_df[to_scale] = scaler.transform(input_df[to_scale])
    prob = stacking_model.predict_proba(input_df)[0][1]
    if prob > 0.7:
        risk_level = "High Risk"
    elif prob > 0.4:
        risk_level = "Moderate Risk"
    else:
        risk_level = "Low Risk"
    return f"{risk_level} of Heart Disease\nEstimated Risk: {prob * 100:.2f}%"
  except Exception as e:
    print("Exception occurred:", str(e))
    return f"Error: {e}"
inputs = [
    gr.Number(label="Age", value=50),
    gr.Radio([0, 1], label="Sex (0 = Female, 1 = Male)"),
    gr.Dropdown([1, 2, 3, 4], label="Chest Pain Type (cp)"),
    gr.Number(label="Resting Blood Pressure (trestbps)", value=120),
    gr.Number(label="Cholesterol (chol)", value=200),
    gr.Radio([0, 1], label="Fasting Blood Sugar > 120 mg/dl (fbs)"),
    gr.Dropdown([0, 1, 2], label="Resting ECG (restecg)"),
    gr.Number(label="Max Heart Rate (thalach)", value=150),
    gr.Radio([0, 1], label="Exercise Induced Angina (exang)"),
    gr.Number(label="ST Depression (oldpeak)", value=1.0),
    gr.Dropdown([1, 2, 3], label="Slope of ST (slope)"),
    gr.Dropdown([0, 1, 2, 3], label="Number of Major Vessels (ca)"),
    gr.Radio([0, 1], label="Thal: Normal (3.0)?"),
    gr.Radio([0, 1], label="Thal: Fixed Defect (6.0)?"),
    gr.Radio([0, 1], label="Thal: Reversible Defect (7.0)?")
]
output = gr.Textbox(label="Heart Disease Risk Result")
app = gr.Interface(
    fn=predict_heart_disease,
    inputs=inputs,
    outputs=output,
    title="💓 Heart Disease Risk Predictor",
    description="This tool uses a stacked model with Logistic Regression as meta-model to predict your risk of heart disease based on health parameters."
)
app.launch(share=True)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.7/322.7 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.3 MB/s eta 0:00:00
Mounted at /content/drive
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://74d79f8853b1fc67a3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
